In [61]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, TimeDistributed,Dropout
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.callbacks import ModelCheckpoint

import numpy as np
import random
import sys
import io

In [62]:
path= get_file('nietzsche.txt',origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text= io.open(path, encoding='utf-8').read().lower()
print ('corpus length: ', len(text))

corpus length:  600893


In [63]:
chars= sorted(list(set(text)))
print ('total chars: ', len(chars))

total chars:  57


In [64]:
print (chars)

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'æ', 'é', 'ë']


In [65]:
char_indices= dict((c,i) for i,c in enumerate(chars))
indices_char= dict((i,c) for i,c in enumerate(chars))

In [66]:
max_len=40
step=3
sentences=[]
next_chars=[]
for i in range(0,len(text)-max_len, step):
    sentences.append(text[i:i+max_len])
    next_chars.append(text[i+max_len])
print ('nb sequences: ', len(sentences))

nb sequences:  200285


In [67]:
print ('Vectorization....')
x=np.zeros((len(sentences), max_len, len(chars)), dtype=np.bool)
y=np.zeros((len(sentences),len(chars)),dtype=np.bool)
print (x.shape,y.shape)

Vectorization....
(200285, 40, 57) (200285, 57)


In [68]:
for i,sentence in enumerate(sentences):
    for t,char in  enumerate(sentence):
        x[i,t,char_indices[char]]=1
    y[i,char_indices[next_chars[i]]] = 1


In [76]:
model=Sequential()
model.add(LSTM(512, input_shape=(max_len,len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
# optimizer= RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 128)               95232     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 57)                7353      
_________________________________________________________________
activation_6 (Activation)    (None, 57)                0         
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


In [77]:
def sample(preds,temperature=1):
    preds=np.asarray(preds).astype('float64')
    preds=np.log(preds)/temperature
#     print (preds)
    exp_preds=np.exp(preds)
#     print (exp_preds)
    probas=np.random.multinomial(1,preds,1)
#     print (probas)
    return np.argmax(probas)

In [79]:
checkpoint = ModelCheckpoint('weights.{epoch:02d}-{loss:.5f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
model.fit(x,y, batch_size=128, epochs=1, callbacks=[checkpoint])
start_index= random.randint(0,len(text)-max_len-1)

generated=''
sentence=text[start_index:start_index+max_len]
generated+=sentence
print ('Generated with seed: "'+sentence + '"')
sys.stdout.write(generated)

for i in range(400):
    x_pred= np.zeros((1,max_len,len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0,t,char_indices[char]]=1.

    preds=model.predict(x_pred, verbose=0)[0]
    next_index= np.argmax(preds)
    next_char=indices_char[next_index]

    generated+=next_char
    sentence=sentence[1:]+next_char
    sys.stdout.write("~")
    sys.stdout.write(next_char)
    sys.stdout.flush()
print()

Epoch 1/1
200285/200285 [==============================] - 99s 495us/step - loss: 2.1316
Generated with seed: "e have poor talents for it. one
may make"
e have poor talents for it. one
may make~ ~t~h~e~ ~s~o~m~e~ 

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:403: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e~ ~s~o~m~e~ ~t~h~e


In [ ]:
checkpoint = ModelCheckpoint('weights.{epoch:02d}-{loss:.5f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
model.fit(x,y, batch_size=128, epochs=1, callbacks=[checkpoint])
start_index= random.randint(0,len(text)-max_len-1)

for diversity in [0.2,0.5,1.0,1.2]:
    print('Diversity: ',diversity)
    generated=''
    sentence=text[start_index:start_index+max_len]
    generated+=sentence
    print ('Generated with seed: "'+sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred= np.zeros((1,max_len,len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0,t,char_indices[char]]=1.

        preds=model.predict(x_pred, verbose=0)[0]
        next_index= sample(preds, diversity)
        next_char=indices_char[next_index]

        generated+=next_char
        sentence=sentence[1:]+next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
for i in range(1,50):
    print('-'*50)
    print ('iteration: ',i)
    checkpoint = ModelCheckpoint('weights.{epoch:02d}-{loss:.5f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
    model.fit(x,y, batch_size=128, epochs=1,callbacks=[checkpoint])
    start_index= random.randint(0,len(text)-max_len-1)
    
    for diversity in [0.2,0.5,1.0,1.2]:
        print('Diversity: ',diversity)
        generated=''
        sentence=text[start_index:start_index+max_len]
        generated+=sentence
        print ('Generated with seed: "'+sentence + '"')
        sys.stdout.write(generated)
        
        for i in range(400):
            x_pred= np.zeros((1,max_len,len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0,t,char_indices[char]]=1.
            
            preds=model.predict(x_pred, verbose=0)[0]
            next_index= sample(preds, diversity)
            next_char=indices_char[next_index]
            
            generated+=next_char
            sentence=sentence[1:]+next_char
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()